In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pii-detection-removal-from-educational-data/sample_submission.csv
/kaggle/input/pii-detection-removal-from-educational-data/train.json
/kaggle/input/pii-detection-removal-from-educational-data/test.json


In [39]:
from transformers import AutoTokenizer

tokenizer_gpt2 = AutoTokenizer.from_pretrained('gpt2')
'''
transformers 라이브러리의 tokenizer 객체는 텍스트를 토큰화한 후, 여러 형태의 정보를 포함하는 딕셔너리를 반환합니다. 
이 딕셔너리 내에서 input_ids는 토큰화된 텍스트의 각 토큰에 대응하는 고유한 숫자 ID들의 리스트입니다.
'''
tokenizer_gpt2("creative")

{'input_ids': [20123, 425], 'attention_mask': [1, 1]}

In [4]:
import json


train_file_path = '/kaggle/input/pii-detection-removal-from-educational-data/train.json'

with open(train_file_path, 'r') as file:
    train = json.load(file)

def retokenize(tokens_org, labels_org, trailing_whitespaces, tokenizer):
    '''
    기존 데이터셋의 trailing_whitespaces 정보를 이용해
    원본토큰 앞에 공백이 있는 경우, 공백을 추가한 후
    공백이 추가된 토큰을 토크나이저에 전달해 토크나이징한 뒤
    결과로 나온 토큰ID 리스트인 new_tokens를 transformed_tokens 리스트에 추가하고
    동시에 해당 토큰의 개수만큼 기존 레이블을 복제해 ransformed_labels 리스트에 추가하는 함수
    '''
    transformed_tokens = []
    transformed_labels = []

    lws = False # Leading whitespace
    '''

    각 토큰 앞에 공백이 있을 경우 (즉, 해당 토큰이 새로운 단어의 시작을 나타내는 경우) 그 공백을 추가하고, 
    이렇게 조정된 텍스트를 토크나이저에 전달하여 토큰화한 뒤, 결과로 나온 토큰 ID들(input_ids)을 new_tokens 변수에 저장
    '''
    for token, label, tws in zip(tokens_org, labels_org, trailing_whitespaces):
        new_tokens = tokenizer((' ' if lws else '') + token)['input_ids']
        transformed_tokens += new_tokens
        transformed_labels += [label] * len(new_tokens)
        lws = tws

    return transformed_tokens, transformed_labels

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
full_text = train[0]['full_text']
trailing_whitespace = train[0]['trailing_whitespace']
original_tokens, original_labels = train[0]['tokens'], train[0]['labels']

In [6]:
new_tokens, new_labels = retokenize(
    original_tokens, 
    original_labels, 
    trailing_whitespace, 
    tokenizer_gpt2)

### 원본 토큰과 retokenize 함수 결과 비교

In [51]:
print("원본토큰", end=" ")
for token, label in zip(original_tokens[:8], original_labels[:8]):
    print(token, label, "|", end= " ")
print(); print("재토큰화", end=" ")    
retokenized_tokens = tokenizer_gpt2.convert_ids_to_tokens(new_tokens[:10])
for new_token, new_label in zip(retokenized_tokens, new_labels[:10]):
    print(new_token, new_label, "|", end= " ")

원본토큰 Design O | Thinking O | for O | innovation O | reflexion O | - O | Avril O | 2021 O | 
재토큰화 Design O | ĠThinking O | Ġfor O | Ġinnovation O | Ġreflex O | ion O | - O | Av O | ril O | Ġ2021 O | 

### retokenize 함수결과와 gpt2 오리지널 토크나이저 결과 비교

retokinze 함수를 쓰면 원본토큰 중 \n\n 이 ĊĊ 으로 토크나이징 되는데  
full_text를 그대로 gpt2 토크나이저에 넣어 토크나이징 하면 \n\n 이 분리되어 별도의 Ċ로 토크나이징 된다.  

In [76]:
'''
Ġ 문자는 GPT-2 토큰화에서 사용되는 특수 문자로, 단어의 시작을 나타내며 앞에 공백이 있음을 의미
Ċ 문자는 \n 줄바꿈을 의미
'''
print(tokenizer_gpt2.convert_ids_to_tokens(198))
tokenizer_gpt2.decode([198])

Ċ


'\n'

In [78]:
fully_tokenized_gpt2 = tokenizer_gpt2(full_text)['input_ids']
fully_retokenized_tokens = tokenizer_gpt2.convert_ids_to_tokens(fully_tokenized_gpt2[11:20])
retokenized_tokens = tokenizer_gpt2.convert_ids_to_tokens(new_tokens[11:20])
for new_token, full_token in zip(retokenized_tokens, fully_retokenized_tokens):
    print(new_token, full_token)

N N
ath ath
al al
ie ie
ĠSy ĠSy
lla lla
ĊĊ Ċ
Chall Ċ
enge Chall


In [64]:
transformed_text = tokenizer_gpt2.decode(new_tokens)
print(f'original_text:\n\n{transformed_text[:89]}')

original_text:

Design Thinking for innovation reflexion-Avril 2021-Nathalie Sylla

Challenge & selection


In [68]:
print(f'full_text:\n')
full_text[:89]

full_text:



'Design Thinking for innovation reflexion-Avril 2021-Nathalie Sylla\n\nChallenge & selection'

### 토크나이징 결과 비교

In [25]:
print("Num tokens originally:", len(original_tokens))
print("Num tokens fully gpt2 tokenized:", len(fully_tokenized_gpt2))
print("Num tokens token-by-token gpt2 tokenized:", len(new_tokens))

Num tokens originally: 753
Num tokens fully gpt2 tokenized: 833
Num tokens token-by-token gpt2 tokenized: 849
